In [3]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Load spending polygons (GeoDataFrame with 'geometry' and 'spending' columns)
spending_polygons = gpd.read_file('../electoral_sections_mc/output_file.geojson')

# Load existing pharmacies (GeoDataFrame with 'geometry' column)
existing_pharmacies = gpd.read_file('farmacias_nombres_limpios.csv')


In [5]:
print(type(spending_polygons))  # Should output: <class 'geopandas.geodataframe.GeoDataFrame'>
print(type(existing_pharmacies))  # Should output: <class 'geopandas.geodataframe.GeoDataFrame'>

<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [7]:
existing_pharmacies.columns

Index(['id', 'clee', 'nom_estab', 'raz_social', 'codigo_act', 'nombre_act',
       'per_ocu', 'tipo_vial', 'nom_vial', 'tipo_v_e_1', 'nom_v_e_1',
       'tipo_v_e_2', 'nom_v_e_2', 'tipo_v_e_3', 'nom_v_e_3', 'numero_ext',
       'letra_ext', 'edificio', 'edificio_e', 'numero_int', 'letra_int',
       'tipo_asent', 'nomb_asent', 'tipoCenCom', 'nom_CenCom', 'num_local',
       'cod_postal', 'cve_ent', 'entidad', 'cve_mun', 'municipio', 'cve_loc',
       'localidad', 'ageb', 'manzana', 'telefono', 'correoelec', 'www',
       'tipoUniEco', 'latitud', 'longitud', 'fecha_alta', 'tipo_farmacia'],
      dtype='object')

In [8]:
existing_pharmacies = existing_pharmacies.rename(columns={"longitud": "longitude", "latitud": "latitude"})

In [9]:
from shapely.geometry import Point

# Assuming the columns are now correctly labeled as 'longitude' and 'latitude'
if 'geometry' not in existing_pharmacies.columns:
    existing_pharmacies['geometry'] = gpd.points_from_xy(existing_pharmacies['longitude'], existing_pharmacies['latitude'])

In [10]:
existing_pharmacies = gpd.GeoDataFrame(existing_pharmacies, geometry='geometry', crs="EPSG:4326")

In [11]:

spending_polygons = spending_polygons.to_crs('EPSG:4326')
existing_pharmacies = existing_pharmacies.to_crs('EPSG:4326')

In [13]:
from deap import base, creator, tools, algorithms
import numpy as np
import random

In [37]:
creator.create("FitnessMulti", base.Fitness, weights=(1.0, 1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

/home/lornarthebreton/miniconda3/envs/dataton_2024/lib/python3.12/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/lornarthebreton/miniconda3/envs/dataton_2024/lib/python3.12/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [38]:
from shapely.geometry import Polygon

# Get bounds of the area for random point generation
minx, miny, maxx, maxy = spending_polygons.total_bounds

def generate_random_point_in_polygon(polygon):
    minx, miny, maxx, maxy = polygon.bounds
    while True:
        p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(p):
            return p

def create_individual():
    individual = []
    for _ in range(200):
        # Randomly select a polygon weighted by spending
        weights = spending_polygons['mean_spending']
        selected_polygon = spending_polygons.sample(weights=weights).iloc[0]['geometry']
        # Generate a random point within the selected polygon
        point = generate_random_point_in_polygon(selected_polygon)
        individual.append((point.x, point.y))
    return creator.Individual(individual)


In [39]:
toolbox = base.Toolbox()
toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [40]:
spending_polygons

,AGEB,CVE_ENT,CVE_MUN,AMBITO,ubica_geo,POBTOT,shape,loc,scale,mean_spending,lower_bound,upper_bound,geometry
0,0372,01,002,Urbana,1002,96.0,2.036582,0.841092,18.501459,30275.425765,76.288045,2.180657e+05,"POLYGON ((-102.07947 22.08552, -102.0807 22.08..."
1,025A,01,002,Urbana,1002,215.0,2.036582,0.841092,18.501459,65293.532045,176.054558,4.560718e+05,"POLYGON ((-102.01512 22.19228, -102.01552 22.1..."
2,0122,01,002,Urbana,1002,1425.0,2.036582,0.841092,18.501459,461355.904614,1081.209715,3.482264e+06,"POLYGON ((-102.08767 22.23623, -102.08751 22.2..."
3,0387,01,002,Urbana,1002,82.0,2.036582,0.841092,18.501459,28028.753781,66.506574,1.880802e+05,"POLYGON ((-102.07481 22.08506, -102.07415 22.0..."
4,0353,01,002,Urbana,1002,137.0,2.036582,0.841092,18.501459,48878.042653,109.732806,3.193725e+05,"POLYGON ((-102.01394 22.1957, -102.01431 22.19..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82275,0025,32,056,Rural,32056,2753.0,1.743633,0.944591,17.974929,575728.769004,4346.256120,3.743536e+06,"POLYGON ((-102.64835 22.81815, -102.64839 22.8..."
82276,0145,32,057,Rural,32057,NaN,NaN,NaN,NaN,639944.174275,2813.950344,4.347291e+06,"POLYGON ((-102.37403 22.74431, -102.37328 22.7..."
82277,015A,32,057,Rural,32057,NaN,NaN,NaN,NaN,639944.174275,2813.950344,4.347291e+06,"POLYGON ((-102.27795 22.76563, -102.27846 22.7..."
82278,0049,32,058,Rural,32058,NaN,NaN,NaN,NaN,639944.174275,2813.950344,4.347291e+06,"POLYGON ((-103.38249 21.5745, -103.38174 21.57..."


In [41]:
from shapely.ops import nearest_points
from shapely.geometry import MultiPoint

def evaluate(individual):
    # Convert individual to GeoDataFrame
    new_pharmacies = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lon, lat in individual], crs='EPSG:4326')
    
    # Calculate Revenue
    joined = gpd.sjoin(new_pharmacies, spending_polygons, how='left', predicate='within')
    total_revenue = joined['mean_spending'].sum()
    
    # Calculate Distance
    all_pharmacies = pd.concat([existing_pharmacies, new_pharmacies], ignore_index=True)
    all_pharm_points = all_pharmacies.geometry.tolist()
    all_points = MultiPoint(all_pharm_points)
    
    # Compute pairwise distances
    min_distance = np.inf
    for i, point in enumerate(all_pharm_points):
        for other_point in all_pharm_points[i+1:]:
            distance = point.distance(other_point)
            if distance < min_distance:
                min_distance = distance
    
    # Normalize objectives
    # For demonstration, we can assume max_possible_revenue and max_possible_distance
    normalized_revenue = total_revenue  # No need to normalize if weights are equal
    normalized_distance = min_distance  # Ditto
    
    # Return negative values because DEAP maximizes by default
    return normalized_revenue, normalized_distance

In [42]:
toolbox.register("select", tools.selNSGA2)

In [43]:
def cx_swap(ind1, ind2):
    """Custom crossover that swaps random subsets of genes."""
    size = min(len(ind1), len(ind2))
    cxpoint1 = random.randint(1, size - 1)
    ind1[cxpoint1:], ind2[cxpoint1:] = ind2[cxpoint1:], ind1[cxpoint1:]
    return ind1, ind2

toolbox.register("mate", cx_swap)

In [44]:
def mut_gaussian(individual, mu=0, sigma=0.01, indpb=0.1):
    """Gaussian mutation for coordinate adjustment."""
    for i in range(len(individual)):
        if random.random() < indpb:
            lon, lat = individual[i]
            lon += random.gauss(mu, sigma)
            lat += random.gauss(mu, sigma)
            # Ensure the point remains within the area
            point = Point(lon, lat)
            if spending_polygons.unary_union.contains(point):
                individual[i] = (lon, lat)
    return individual,

toolbox.register("mutate", mut_gaussian)

In [45]:
# Parameters
POPULATION_SIZE = 10
NGEN = 10
CXPB = 0.7  # Crossover probability
MUTPB = 0.2  # Mutation probability

# Register functions
toolbox.register("evaluate", evaluate)

In [46]:
population = toolbox.population(n=POPULATION_SIZE)

In [47]:
hof = tools.HallOfFame(1)

In [51]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

In [52]:
algorithms.eaMuPlusLambda(population, toolbox, mu=POPULATION_SIZE, 
                          lambda_=POPULATION_SIZE, cxpb=CXPB, mutpb=MUTPB, 
                          ngen=NGEN, stats=stats, halloffame=hof, verbose=True)

KeyboardInterrupt: 